In [2]:
import pickle
import numpy as np
import pandas as pd
import sklearn.metrics as sm
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn import preprocessing
from sklearn.metrics import pairwise
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
import seaborn as sns
from pandas.plotting import parallel_coordinates

In [3]:
data = pd.read_csv("inec_encuesta-estructural-emprresarial_establecimientos_2019.csv", delimiter=';')
data.sample(15)

,id_empresa,tipo_establecimiento_cod,tipo_establecimiento,provincia_cod,provincia_desc,ciiu_cod,ciiu_desc,personal_ocupado,sueldos,ventas
428,13601962017,Auxiliar,Auxiliar,EL ORO,EL ORO,,,0,0,0
19659,14819119172,Sucursal,Sucursal,PICHINCHA,PICHINCHA,"VENTA AL POR MENOR DE PRENDAS DE VESTIR, CALZA...","Venta al por menor de prendas de vestir, calza...",4,26390,245182
14412,13829271171,Sucursal,Sucursal,PICHINCHA,PICHINCHA,"VENTA AL POR MENOR DE PRENDAS DE VESTIR, CALZA...","Venta al por menor de prendas de vestir, calza...",5,28170,520806
9053,13824564170,Auxiliar,Auxiliar,PICHINCHA,PICHINCHA,,,17,186060,0
9420,13824714171,Sucursal,Sucursal,GUAYAS,GUAYAS,ACTIVIDADES DE FOTOGRAFÍA.,Actividades de fotografía.,5,28751,138437
6078,13707900096,Sucursal,Sucursal,GUAYAS,GUAYAS,"CAPTACIÓN, TRATAMIENTO Y DISTRIBUCIÓN DE AGUA.","Captación, tratamiento y distribución de agua.",0,0,0
18329,14776409171,Sucursal,Sucursal,IMBABURA,IMBABURA,VENTA AL POR MENOR DE EQUIPO DE SONIDO Y VÍDEO...,Venta al por menor de equipo de sonido y vídeo...,0,0,0
16916,14678862097,Sucursal,Sucursal,GUAYAS,GUAYAS,ACTIVIDADES DE MÉDICOS Y ODONTÓLOGOS.,Actividades de médicos y odontólogos.,10,176362,2592363
21989,46564850098,Sucursal,Sucursal,GUAYAS,GUAYAS,ACTIVIDADES DE RESTAURANTES Y DE SERVICIO MÓVI...,Actividades de restaurantes y de servicio móvi...,10,35128,32994
22190,46618181090,Sucursal,Sucursal,EL ORO,EL ORO,"GENERACIÓN, TRANSMISIÓN Y DISTRIBUCIÓN DE ENER...","Generación, transmisión y distribución de ener...",0,0,0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23524 entries, 0 to 23523
Data columns (total 10 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   id_empresa                23524 non-null  int64 
 1   tipo_establecimiento_cod  23524 non-null  object
 2   tipo_establecimiento      23524 non-null  object
 3   provincia_cod             23524 non-null  object
 4   provincia_desc            23524 non-null  object
 5   ciiu_cod                  23524 non-null  object
 6   ciiu_desc                 23524 non-null  object
 7   personal_ocupado          23524 non-null  int64 
 8   sueldos                   23524 non-null  int64 
 9   ventas                    23524 non-null  int64 
dtypes: int64(4), object(6)
memory usage: 1.8+ MB


In [5]:
data.shape

(23524, 10)

In [6]:
data.drop(columns=['id_empresa'], inplace=True) 

In [7]:
data.columns

Index(['tipo_establecimiento_cod', 'tipo_establecimiento', 'provincia_cod',
       'provincia_desc', 'ciiu_cod', 'ciiu_desc', 'personal_ocupado',
       'sueldos', 'ventas'],
      dtype='object')

In [8]:
# Reemplaza los espacios en blanco (' ') con NaN
#data.replace(' ', np.nan, inplace=True)

# Reemplaza las cadenas vacías ('') con NaN
#data.replace('', np.nan, inplace=True)

# Elimina las filas con al menos un valor faltante (NaN)
#data_cleaned = data.dropna().copy()

# Imprime el número de filas antes y después de la limpieza
#print("Número de filas antes de limpieza:", len(data))
#print("Número de filas después de limpieza:", len(data_cleaned))

In [9]:
# Elimina las filas en las que al menos una columna contenga un espacio en blanco o cadena vacía
data_cleaned = data[~(data == ' ').any(axis=1) & ~(data == '').any(axis=1)].copy()

# Imprime el número de filas antes y después de la eliminación
print("Número de filas antes de eliminación:", len(data))
print("Número de filas después de limpieza:", len(data_cleaned))

Número de filas antes de eliminación: 23524
Número de filas después de limpieza: 21695


In [10]:
# Aplica strip() a todas las celdas del DataFrame para eliminar espacios en blanco
data = data.applymap(lambda x: x.strip() if isinstance(x, str) else x)

In [11]:
data.head()

,tipo_establecimiento_cod,tipo_establecimiento,provincia_cod,provincia_desc,ciiu_cod,ciiu_desc,personal_ocupado,sueldos,ventas
0,Matriz,Matriz,AZUAY,AZUAY,"VENTA AL POR MAYOR DE ALIMENTOS, BEBIDAS Y TAB...","Venta al por mayor de alimentos, bebidas y tab...",69,600616,9913036
1,Auxiliar,Auxiliar,AZUAY,AZUAY,,,0,0,0
2,Único,Único,AZUAY,AZUAY,VENTA AL POR MENOR DE OTROS PRODUCTOS NUEVOS E...,Venta al por menor de otros productos nuevos e...,7,66000,2117598
3,Único,Único,AZUAY,AZUAY,VENTA AL POR MENOR DE COMBUSTIBLES PARA VEHÍCU...,Venta al por menor de combustibles para vehícu...,30,298257,6664329
4,Único,Único,AZUAY,AZUAY,FABRICACIÓN DE PRODUCTOS DE PLÁSTICO.,Fabricación de productos de plástico.,45,319010,1724784


In [12]:
data_cleaned.head()

,tipo_establecimiento_cod,tipo_establecimiento,provincia_cod,provincia_desc,ciiu_cod,ciiu_desc,personal_ocupado,sueldos,ventas
0,Matriz,Matriz,AZUAY,AZUAY,"VENTA AL POR MAYOR DE ALIMENTOS, BEBIDAS Y TAB...","Venta al por mayor de alimentos, bebidas y tab...",69,600616,9913036
2,Único,Único,AZUAY,AZUAY,VENTA AL POR MENOR DE OTROS PRODUCTOS NUEVOS E...,Venta al por menor de otros productos nuevos e...,7,66000,2117598
3,Único,Único,AZUAY,AZUAY,VENTA AL POR MENOR DE COMBUSTIBLES PARA VEHÍCU...,Venta al por menor de combustibles para vehícu...,30,298257,6664329
4,Único,Único,AZUAY,AZUAY,FABRICACIÓN DE PRODUCTOS DE PLÁSTICO.,Fabricación de productos de plástico.,45,319010,1724784
5,Único,Único,AZUAY,AZUAY,ACTIVIDADES DE RESTAURANTES Y DE SERVICIO MÓVI...,Actividades de restaurantes y de servicio móvi...,24,193291,1300985


In [13]:
# Elimina las filas con al menos un valor faltante (NaN)
#data_cleaned = data.dropna().copy()

In [13]:
#missing_values_count = data.isna().sum()
#print(missing_values_count)

In [14]:
data_cleaned['ciiu_cod'].value_counts()

VENTA AL POR MENOR DE PRODUCTOS FARMACÉUTICOS Y MEDICINALES, COSMÉTICOS Y ARTÍCULOS DE TOCADOR EN COMERCIOS ESPECIALI    2582
VENTA AL POR MAYOR DE OTROS ENSERES DOMÉSTICOS.                                                                          1151
ACTIVIDADES DE RESTAURANTES Y DE SERVICIO MÓVIL DE COMIDAS.                                                              1125
VENTA AL POR MENOR DE APARATOS ELÉCTRICOS DE USO DOMÉSTICO, MUEBLES, EQUIPO DE ILUMINACIÓN Y OTROS ENSERES DOMÉSTICO      896
VENTA AL POR MENOR EN COMERCIOS NO ESPECIALIZADOS CON PREDOMINIO DE LA VENTA DE ALIMENTOS, BEBIDAS O TABACO.              771
                                                                                                                         ... 
OTRAS ACTIVIDADES DE TECNOLOGÍA DE LA INFORMACIÓN Y DE SERVICIOS INFORMÁTICOS.                                              1
FABRICACIÓN DE GAS; DISTRIBUCIÓN DE COMBUSTIBLES GASEOSOS POR TUBERÍAS.                                               

In [15]:
data_cleaned['ciiu_desc'].value_counts()

Venta al por menor de productos farmacéuticos y medicinales, cosméticos y artículos de tocador en comercios especializados.                           2582
Venta al por mayor de otros enseres domésticos.                                                                                                       1151
Actividades de restaurantes y de servicio móvil de comidas.                                                                                           1125
Venta al por menor de aparatos eléctricos de uso doméstico, muebles, equipo de iluminación y otros enseres domésticos en comercios especializados.     896
Venta al por menor en comercios no especializados con predominio de la venta de alimentos, bebidas o tabaco.                                           771
                                                                                                                                                      ... 
Otras actividades de tecnología de la información y de servicios infor

In [14]:
data_cleaned['provincia_desc'].value_counts()

GUAYAS                           6578
PICHINCHA                        6158
MANABÍ                           1553
AZUAY                            1281
EL ORO                            820
LOS RÍOS                          766
TUNGURAHUA                        686
STO. DOMINGO DE LOS TSÁCHILAS     644
IMBABURA                          453
LOJA                              388
ESMERALDAS                        346
CHIMBORAZO                        296
SANTA ELENA                       276
COTOPAXI                          262
ORELLANA                          214
SUCUMBÍOS                         214
CAÑAR                             175
CARCHI                            126
BOLÍVAR                            90
ZAMORA CHINCHIPE                   77
MORONA SANTIAGO                    76
GALÁPAGOS                          73
PASTAZA                            72
NAPO                               71
Name: provincia_desc, dtype: int64

In [15]:
data_cleaned['provincia_cod'].value_counts()

GUAYAS                           6578
PICHINCHA                        6158
MANABI                           1553
AZUAY                            1281
EL ORO                            820
LOS RIOS                          766
TUNGURAHUA                        686
STO. DOMINGO DE LOS TSACHILAS     644
IMBABURA                          453
LOJA                              388
ESMERALDAS                        346
CHIMBORAZO                        296
SANTA ELENA                       276
COTOPAXI                          262
ORELLANA                          214
SUCUMBIOS                         214
CAÑAR                             175
CARCHI                            126
BOLIVAR                            90
ZAMORA CHINCHIPE                   77
MORONA SANTIAGO                    76
GALAPAGOS                          73
PASTAZA                            72
NAPO                               71
Name: provincia_cod, dtype: int64

In [16]:
data_cleaned['tipo_establecimiento_cod'].value_counts()

Sucursal    17916
Matriz       2447
Único        1332
Name: tipo_establecimiento_cod, dtype: int64

In [17]:
data_cleaned['tipo_establecimiento'].value_counts()

Sucursal    17916
Matriz       2447
Único        1332
Name: tipo_establecimiento, dtype: int64

In [18]:
data_cleaned.drop(columns=['ciiu_cod', 'tipo_establecimiento_cod', 'provincia_cod'], inplace=True) 

In [19]:
data_cleaned.columns

Index(['tipo_establecimiento', 'provincia_desc', 'ciiu_desc',
       'personal_ocupado', 'sueldos', 'ventas'],
      dtype='object')

In [20]:
data.sample(10)
data_cleaned.sample(5)

,tipo_establecimiento,provincia_desc,ciiu_desc,personal_ocupado,sueldos,ventas
21576,Sucursal,PICHINCHA,Actividades de telecomunicaciones alámbrica.,1,6987,39898
2854,Sucursal,LOS RÍOS,Venta al por menor en comercios no especializa...,22,112079,1773209
13354,Sucursal,SUCUMBÍOS,Actividades de consultoría de gestión.,7,56348,412723
22059,Sucursal,GUAYAS,"Generación, transmisión y distribución de ener...",0,0,0
19019,Sucursal,GUAYAS,Otras actividades de venta al por menor no rea...,0,0,124698


In [21]:
# Convertir las variables categóricas a tipo 'category'
data_cleaned['tipo_establecimiento'] = data_cleaned['tipo_establecimiento'].astype('category')
data_cleaned['provincia_desc'] = data_cleaned['provincia_desc'].astype('category')
data_cleaned['ciiu_desc'] = data_cleaned['ciiu_desc'].astype('category')

# Definir nuevas asignaciones de categorías basadas en los resultados de value_counts()
provincia_desc_categories = {
'GUAYAS': 0,
'PICHINCHA': 1,
'MANABÍ': 3,
'AZUAY': 4,
'EL ORO': 5,
'LOS RÍOS': 6,
'TUNGURAHUA': 7,
'STO. DOMINGO DE LOS TSÁCHILAS': 8,
'IMBABURA': 9,
'LOJA': 10,
'ESMERALDAS': 11,
'CHIMBORAZO': 12,
'SANTA ELENA': 13,
'COTOPAXI': 14,
'ORELLANA': 15,
'SUCUMBÍOS': 16,
'CAÑAR': 17,
'CARCHI': 18,
'BOLÍVAR': 19,
'ZAMORA CHINCHIPE': 20,
'MORONA SANTIAGO': 21,
'GALÁPAGOS': 22,
'PASTAZA': 23,
'NAPO': 24,
}

###############################################
###############################################

# Obtener los valores únicos de tipo_establecimiento
unique_tipo_establecimiento = data_cleaned['tipo_establecimiento'].unique()

# Crear el diccionario 'tipo_establecimientor_categories' usando un bucle for
tipo_establecimientor_categories = {}
for index, tipo_establecimiento in enumerate(unique_tipo_establecimiento):
    tipo_establecimientor_categories[tipo_establecimiento] = index
    
###############################################
###############################################

# Obtener los valores únicos de tipo_establecimiento
unique_ciiu_desc = data_cleaned['ciiu_desc'].unique()

# Crear el diccionario 'ciiu_desc_categories' usando un bucle for
ciiu_desc_categories = {}
for index, ciiu_desc in enumerate(unique_ciiu_desc):
    ciiu_desc_categories[ciiu_desc] = index

In [22]:
# Renombrar las categorías utilizando los nuevos códigos
data_cleaned['tipo_establecimiento'] = data_cleaned['tipo_establecimiento'].cat.rename_categories(tipo_establecimientor_categories)
data_cleaned['provincia_desc'] = data_cleaned['provincia_desc'].cat.rename_categories(provincia_desc_categories)
data_cleaned['ciiu_desc'] = data_cleaned['ciiu_desc'].cat.rename_categories(ciiu_desc_categories)

# Convertir las variables categóricas a enteros
data_cleaned['tipo_establecimiento'] = data_cleaned['tipo_establecimiento'].astype('int64')
data_cleaned['provincia_desc'] = data_cleaned['provincia_desc'].astype('int64')
data_cleaned['ciiu_desc'] = data_cleaned['ciiu_desc'].astype('int64')

# Mostrar una muestra de los datos transformados
data_cleaned.sample(10)

,tipo_establecimiento,provincia_desc,ciiu_desc,personal_ocupado,sueldos,ventas
17170,2,4,26,4,33692,155265
7854,2,3,10,2,8077,311372
7197,2,10,10,6,42881,523716
19826,2,1,28,0,0,0
13514,2,0,4,14,47300,597017
23121,1,6,65,0,0,4376060
20902,2,4,6,10,169330,176427
133,2,6,11,30,408672,1742298
22397,2,0,10,2,12659,156599
12458,2,1,10,3,39766,425592


In [23]:
# Mostrar una muestra de los datos transformados

#data_cleaned=data_cleaned[data_cleaned['ventas']!=0]
#data_cleaned=data_cleaned[data_cleaned['ventas']>0]
data_cleaned = data_cleaned.reset_index(drop=True)
data_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21695 entries, 0 to 21694
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype
---  ------                --------------  -----
 0   tipo_establecimiento  21695 non-null  int64
 1   provincia_desc        21695 non-null  int64
 2   ciiu_desc             21695 non-null  int64
 3   personal_ocupado      21695 non-null  int64
 4   sueldos               21695 non-null  int64
 5   ventas                21695 non-null  int64
dtypes: int64(6)
memory usage: 1017.1 KB


In [38]:
#print((data_cleaned['ventas']==0).sum())
print((data_cleaned['ventas']<=0).sum())
# Calcula el Q1 y el Q3
Q1 = data_cleaned['sueldos'].quantile(0.25)
Q3 = data_cleaned['sueldos'].quantile(0.75)

# Calcula el IQR
IQR = Q3 - Q1

# Define un umbral para los valores atípicos
umbral_superior = Q3 + 1.5 * IQR
umbral_inferior = Q1 - 1.5 * IQR

# Elimina las filas con valores de sueldos por encima o por debajo del umbral
data_cleaned = data_cleaned[(data_cleaned['sueldos'] >= umbral_inferior) & (data_cleaned['sueldos'] <= umbral_superior)]
data_cleaned.info()

3905
<class 'pandas.core.frame.DataFrame'>
Int64Index: 18609 entries, 1 to 21694
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype
---  ------                --------------  -----
 0   tipo_establecimiento  18609 non-null  int64
 1   provincia_desc        18609 non-null  int64
 2   ciiu_desc             18609 non-null  int64
 3   personal_ocupado      18609 non-null  int64
 4   sueldos               18609 non-null  int64
 5   ventas                18609 non-null  int64
dtypes: int64(6)
memory usage: 1017.7 KB


In [39]:

Q1 = data_cleaned['ventas'].quantile(0.25)
Q3 = data_cleaned['ventas'].quantile(0.75)

# Calcula el IQR
IQR = Q3 - Q1

# Define un umbral para los valores atípicos
umbral_superior = Q3 + 1.5 * IQR
umbral_inferior = Q1 - 1.5 * IQR

# Elimina las filas con valores de sueldos por encima o por debajo del umbral
data_cleaned = data_cleaned[(data_cleaned['ventas'] >= umbral_inferior) & (data_cleaned['ventas'] <= umbral_superior)]
data_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16175 entries, 1 to 21693
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype
---  ------                --------------  -----
 0   tipo_establecimiento  16175 non-null  int64
 1   provincia_desc        16175 non-null  int64
 2   ciiu_desc             16175 non-null  int64
 3   personal_ocupado      16175 non-null  int64
 4   sueldos               16175 non-null  int64
 5   ventas                16175 non-null  int64
dtypes: int64(6)
memory usage: 884.6 KB


### ENTRENAMIENTO DEL MODELO (REGRESION POLINOMIAL Y LINEAL)

In [24]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler

In [25]:
scaler = StandardScaler()
data_cleaned_normalized=scaler.fit_transform(data_cleaned.select_dtypes(include=[np.number]))
data_cleaned_normalized=pd.DataFrame(data_cleaned_normalized,columns=data_cleaned.select_dtypes(include=[np.number]).columns)
# Target no normalizar
print(scaler.mean_,scaler.scale_)
y = data_cleaned['sueldos'] #variable target
X = data_cleaned.drop(columns=['sueldos']) #variables input

[1.71302143e+00 3.63452408e+00 5.30510256e+01 3.23527080e+01
 3.71477205e+05 4.98353529e+06] [6.55899230e-01 4.99081960e+00 5.50606213e+01 1.24544877e+02
 1.73395121e+06 5.28045164e+07]


In [26]:
data_cleaned_normalized.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21695 entries, 0 to 21694
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   tipo_establecimiento  21695 non-null  float64
 1   provincia_desc        21695 non-null  float64
 2   ciiu_desc             21695 non-null  float64
 3   personal_ocupado      21695 non-null  float64
 4   sueldos               21695 non-null  float64
 5   ventas                21695 non-null  float64
dtypes: float64(6)
memory usage: 1017.1 KB


In [27]:
data_cleaned_normalized.sample(5)
print(data_cleaned_normalized.iloc[794,:])


tipo_establecimiento    0.437535
provincia_desc          0.473965
ciiu_desc              -0.763722
personal_ocupado       -0.243709
sueldos                -0.202921
ventas                 -0.091552
Name: 794, dtype: float64


- Separamos data de training y testing:

In [51]:
# Asegúrate de que 'test_data' esté en el mismo formato que los datos de entrenamiento
test_data = [2, 1, 4, 15, 710149]

# Asegúrate de que 'test_data' sea una matriz NumPy bidimensional
test_data_d = np.array(test_data).reshape(1, -1)

# Realiza la predicción
predicted_price = linear_regressor.predict(test_data_d)

print("Predicted price:", predicted_price)

Predicted price: [129862.11287546]


C:\Users\santi\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


In [30]:
X[0:5]

,tipo_establecimiento,provincia_desc,ciiu_desc,personal_ocupado,ventas
0,0,4,0,69,9913036
1,1,4,1,7,2117598
2,1,4,2,30,6664329
3,1,4,3,45,1724784
4,1,4,4,24,1300985


In [31]:
y[0:5]

0    600616
1     66000
2    298257
3    319010
4    193291
Name: sueldos, dtype: int64

In [32]:
data_cleaned_normalized.shape

(21695, 6)

In [33]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=20)

In [34]:
from sklearn import linear_model
#Cree y entrene el modelo de regresor lineal
# Create the linear regressor model
linear_regressor = linear_model.LinearRegression()

In [35]:
# Train the model using the training sets
linear_regressor.fit(X_train, y_train)

LinearRegression()

In [36]:
# Predict the output
y_test_pred = linear_regressor.predict(X_test)

In [37]:
# Calcule los valores para las diferentes métricas, consulte su interpretación
print("Linear regressor performance:")
print("Mean absolute error =", round(sm.mean_absolute_error(y_test, y_test_pred), 2))
print("Mean squared error =", round(sm.mean_squared_error(y_test, y_test_pred), 2)) 
print("Median absolute error =", round(sm.median_absolute_error(y_test, y_test_pred), 2)) 
print("Explain variance score =", round(sm.explained_variance_score(y_test, y_test_pred), 2))
print("R2 score =", round(sm.r2_score(y_test, y_test_pred), 2))

Linear regressor performance:
Mean absolute error = 152721.72
Mean squared error = 422914277966.87
Median absolute error = 30704.72
Explain variance score = 0.76
R2 score = 0.76


In [39]:
with open('trained_model.pkl','wb') as file:
    pickle.dump(linear_regressor,file)

In [38]:
# Train the model using the training sets
polynomial = PolynomialFeatures(degree=2, include_bias=False)

In [39]:
X_train_transformed = polynomial.fit_transform(X_train) #con la data de entrenamiento se configura el polinomio
datapoint = [[1, 3, 1, 67, 19036]]
poly_datapoint = polynomial.fit_transform(datapoint)

In [40]:
polynomial.degree

2

In [41]:
polynomial.get_feature_names_out()

array(['x0', 'x1', 'x2', 'x3', 'x4', 'x0^2', 'x0 x1', 'x0 x2', 'x0 x3',
       'x0 x4', 'x1^2', 'x1 x2', 'x1 x3', 'x1 x4', 'x2^2', 'x2 x3',
       'x2 x4', 'x3^2', 'x3 x4', 'x4^2'], dtype=object)

In [42]:
poly_datapoint

array([[1.00000000e+00, 3.00000000e+00, 1.00000000e+00, 6.70000000e+01,
        1.90360000e+04, 1.00000000e+00, 3.00000000e+00, 1.00000000e+00,
        6.70000000e+01, 1.90360000e+04, 9.00000000e+00, 3.00000000e+00,
        2.01000000e+02, 5.71080000e+04, 1.00000000e+00, 6.70000000e+01,
        1.90360000e+04, 4.48900000e+03, 1.27541200e+06, 3.62369296e+08]])

In [43]:
#Realice la predicción usando regresores lineales y polinómicos para ver la
#diferencia:

poly_linear_model = linear_model.LinearRegression()
poly_linear_model.fit(X_train_transformed, y_train) #ajuste polinomial

print("\nLinear regression:\n", linear_regressor.predict(datapoint))
print("\nPolynomial regression:\n", poly_linear_model.predict(poly_datapoint))


Linear regression:
 [748554.61381973]

Polynomial regression:
 [401777.94284864]


C:\Users\santi\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


## polinomio grado 3

In [44]:
poly = PolynomialFeatures(degree=2, include_bias=False)

In [45]:
poly_features = poly.fit_transform(X)

In [46]:
poly_features

array([[0.00000000e+00, 4.00000000e+00, 0.00000000e+00, ...,
        4.76100000e+03, 6.83999484e+08, 9.82682827e+13],
       [1.00000000e+00, 4.00000000e+00, 1.00000000e+00, ...,
        4.90000000e+01, 1.48231860e+07, 4.48422129e+12],
       [1.00000000e+00, 4.00000000e+00, 2.00000000e+00, ...,
        9.00000000e+02, 1.99929870e+08, 4.44132810e+13],
       ...,
       [1.00000000e+00, 0.00000000e+00, 2.20000000e+01, ...,
        4.41000000e+02, 2.00114775e+08, 9.08070820e+13],
       [0.00000000e+00, 0.00000000e+00, 1.33000000e+02, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [2.00000000e+00, 0.00000000e+00, 1.33000000e+02, ...,
        5.76000000e+02, 8.27656872e+08, 1.18926371e+15]])

In [47]:
X_train, X_test, y_train, y_test = train_test_split(poly_features, y, test_size=0.3, random_state=20)

poly_reg_model = linear_model.LinearRegression()
poly_reg_model.fit(X_train, y_train)
y_predicted = poly_reg_model.predict(X_test)

In [48]:
from sklearn.metrics import mean_squared_error

In [49]:
poly_reg_rmse = np.sqrt(mean_squared_error(y_test, y_predicted))
poly_reg_rmse

1004648.9706940487

In [50]:
# Measure performance
print("Polinomial Regressor performance:")
print("Mean absolute error =", round(sm.mean_absolute_error(y_test, y_predicted), 2))
print("Mean squared error =", round(sm.mean_squared_error(y_test, y_predicted), 2))
print("Median absolute error =", round(sm.median_absolute_error(y_test, y_predicted), 2))
print("Explained variance score =", round(sm.explained_variance_score(y_test, y_predicted), 2))
print("R2 score =", round(sm.r2_score(y_test, y_predicted), 2))

Polinomial Regressor performance:
Mean absolute error = 201878.0
Mean squared error = 1009319554316.61
Median absolute error = 56290.35
Explained variance score = 0.42
R2 score = 0.42


### polinomio grado 2

In [87]:
from sklearn.metrics import mean_squared_error

In [88]:
poly = PolynomialFeatures(degree=2, include_bias=False)

In [89]:
poly_features = poly.fit_transform(X)

In [90]:
X_train, X_test, y_train, y_test = train_test_split(poly_features, y, test_size=0.3, random_state=20)

In [91]:
poly_reg_model = linear_model.LinearRegression()
poly_reg_model.fit(X_train, y_train)
y_predicted = poly_reg_model.predict(X_test)

In [92]:
poly_reg_rmse = np.sqrt(mean_squared_error(y_test, y_predicted))
poly_reg_rmse

52425.926604752785

In [93]:
# Measure performance
print("Linear Regressor performance:")
print("Mean absolute error =", round(sm.mean_absolute_error(y_test, y_predicted), 2))
print("Mean squared error =", round(sm.mean_squared_error(y_test, y_predicted), 2))
print("Median absolute error =", round(sm.median_absolute_error(y_test, y_predicted), 2))
print("Explained variance score =", round(sm.explained_variance_score(y_test, y_predicted), 2))
print("R2 score =", round(sm.r2_score(y_test, y_predicted), 2))

Linear Regressor performance:
Mean absolute error = 27158.88
Mean squared error = 2748477780.37
Median absolute error = 12416.34
Explained variance score = 0.57
R2 score = 0.57


### **SVM regressor**

In [37]:
from sklearn import datasets
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.utils import shuffle

In [38]:
train_X, valid_X, train_y, valid_y = train_test_split(X, y, test_size=0.3, random_state=30)

In [39]:
# Create Support Vector Regression model
sv_regressor = SVR(kernel='linear', C=1, epsilon=0.1)

In [ ]:
# Train Support Vector Regressor
sv_regressor.fit(train_X, train_y)

In [ ]:
# Evaluate performance of Support Vector Regressor
y_test_pred = sv_regressor.predict(valid_X)
mse = mean_squared_error(valid_y, y_test_pred)
r2 = r2_score(valid_y, y_test_pred)
print("\n#### Performance ####")
print("Mean squared error =", round(mse, 2))
print("R2 score =", round(r2, 2))